In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Function to parse the quoted string of values into a list of floats
def parse_velocity_string(velocity_string):
    return list(map(float, velocity_string.strip('"').split(',')))

In [ ]:
# Function to load and process velocity data
def load_velocity_data(x_file, z_file, u_file, w_file):
    x_coords = pd.read_csv(x_file).values.flatten()
    z_coords = pd.read_csv(z_file).values.flatten()

    u_velocities_df = pd.read_csv(u_file, header = None)
    w_velocities_df = pd.read_csv(w_file, header = None)

    u_velocities = u_velocities_df[0].apply(parse_velocity_string).values
    w_velocities = w_velocities_df[0].apply(parse_velocity_string).values

    u_velocities = np.array(u_velocities.tolist())
    w_velocities = np.array(w_velocities.tolist())

    expected_z_size = len(z_coords) + 1
    expected_x_size = len(x_coords) + 1

    if u_velocities.shape != (expected_z_size, expected_x_size):
        raise ValueError(f"u_velocities shape does not match the expected grid shape. Got {u_velocities.shape}, expected {(expected_z_size, expected_x_size)}")
    if w_velocities.shape != (expected_z_size, expected_x_size):
        raise ValueError(f"w_velocities shape does not match the expected grid shape. Got {w_velocities.shape}, expected {(expected_z_size, expected_x_size)}")

    X_grid, Z_grid = np.meshgrid(x_coords, z_coords)
    X_flat = X_grid.flatten()
    Z_flat = Z_grid.flatten()
    X = np.column_stack((X_flat, Z_flat))

    u_flat = u_velocities.flatten()
    w_flat = w_velocities.flatten()
    y = np.column_stack((u_flat, w_flat))

    return X, y

In [ ]:
# Load and process data for Reynolds number 180
X_180, y_180 = load_velocity_data('x_768_128.csv', 'z_768_128.csv', 'U_768_128.csv', 'W_768_128.csv')

# Load and process data for Reynolds number 400
X_400, y_400 = load_velocity_data('x_512_320.csv', 'z_512_320.csv', 'U_512_320.csv', 'W_512_320.csv')

# Combine the datasets
X_combined = np.vstack((X_180, X_400))
y_combined = np.vstack((y_180, y_400))

In [ ]:
# Train the machine learning model
model = Sequential([
    Dense(64, activation  = 'relu', input_shape = (2,)),
    Dense(64, activation = 'relu'),
    Dense(2)
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_combined, y_combined, epochs = 200, batch_size = 32, validation_split = 0.2)

In [ ]:
# Predict velocities for Reynolds number 1000
new_x_coords = pd.read_csv('x1000_768_128.csv').values.flatten()
new_z_coords = pd.read_csv('z1000_768_128.csv').values.flatten()

new_X_grid, new_Z_grid = np.meshgrid(new_x_coords, new_z_coords)
new_X_flat = new_X_grid.flatten()
new_Z_flat = new_Z_grid.flatten()
new_X = np.column_stack((new_X_flat, new_Z_flat))

predicted_velocities = model.predict(new_X)

predicted_u_velocities = predicted_velocities[:, 0]
predicted_w_velocities = predicted_velocities[:, 1]

In [ ]:
# Save the predicted velocities to CSV files
pd.DataFrame(predicted_u_velocities).to_csv('predicted_u_velocities_1000.csv', index = False)
pd.DataFrame(predicted_w_velocities).to_csv('predicted_w_velocities_1000.csv', index = False)